In [1]:
import os
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm 
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from collections import defaultdict
tqdm.pandas()

In [82]:
# 전처리, 명사추출, 키워드추출, 불용어후처리까지 마친 데이터
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_after_cleaning_nickname.csv')
comments_df = pd.read_csv(file_path)


# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df = comments_df.copy()
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
#sample_df['word_list'] = sample_df['word_list'].apply(lambda x: " ".join(x))
sample_df['members'] =  sample_df['members'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [83]:
sample_df.head()

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list,members
0,NMIXX,별별별 (See that),node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,True,"[중독, 진심, 노래, 의문, 엔믹스]",NaN
1,NMIXX,별별별 (See that),node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,True,"[기도, 아쉬움, 대쉬, 금도, 대중성, 노래]",NaN
2,NMIXX,별별별 (See that),node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,True,"[본부장, 엘범, 천만, 클릭, 연락, 연결, 취해, 피씨, 뮤비, 노래, 뷰수, ...",NaN
3,NMIXX,별별별 (See that),node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,True,[노래],NaN
4,NMIXX,별별별 (See that),node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,True,"[걸그룹, 멤버, 빵빵, 노력, 묵자, 얼마나, 세대, 외모췍, 더더, 리더, 해원]",[해원]


In [84]:
group_members = {
    'ITZY': ['예지', '리아', '류진', '채령', '유나'],
    'NewJeans': ['민지', '하니', '다니엘', '해린', '혜인'],
    'aespa': ['카리나', '지젤', '윈터', '닝닝'],
    'LE SSERAFIM': ['김채원', '사쿠라', '허윤진', '카즈하', '홍은채', '김가람'],
    'IVE': ['안유진', '가을', '레이', '장원영', '리즈', '이서'],
    'NMIXX': ['릴리', '해원', '설윤', '배이', '지우', '규진', '지니'],
    'tripleS': ['윤서연', '정혜린', '이지우', '김채연', '김유연', '김수민', '김나경', '공유빈', '카에데', '서다현', '코토네', '곽연지', '니엔', '박소현', '신위', '마유', '린', '주빈', '정하연', '박시온', '김채원', '설린', '서아', '지연'],
    'FIFTY FIFTY': ['키나', '예원', '문샤넬', '하나', '아테나', '새나', '아란', '시오'],
    'STAYC': ['수민', '시은', '아이사', '세은', '윤', '재이'],
    'Kep1er': ['최유진', '히카루', '샤오팅', '휴닝바이에', '김채현', '서영은', '김다연', '마시로', '강예서']
}

# 그룹별 멤버와 인식어 리스트를 정의합니다.
nicknames = {
    'ITZY': {
        '예지': ['예지', '황예지', '옞덩', '예지섭', '깽이', '루시', 'Lucy', '엄예지', '황예술', '황작가', '라이트 퓨리', '예취', '늑냥이', '할머니', '열정예지', '황리더', '예지선배', '예스코', '황무지', '전주 불주먹', '배라예지', '옞또몰', '황드라이버', '갑분예', '예무새', '거울요정', '황 스패로우', '우주대스타', 'i옞D', '예도리', '황미카', '예지공주', '황자몽', '황꾸꾸', '예드립', '스윗리를키리', '아기고양이슈가파우더말랑공쥬', '황소', '표지', '검하단 대장'],
        '리아': ['리아', '최지수', '최리아', '랴', '공주님', '랴무늘보', '러블리아', '롯데리아', '꿀리아', '쌀리아', '시나몬리아', '치코리아', '우리아', '토일리아', '인간뾰루지', '오믈리아', '스트로베리아', '말랑이', '오드리아 햇반', '치즈'],
        '류진': ['류진', '신류진', '르땡', '류숭이', '류세', '류세대마왕', '르', '르깅', '파란머리 걔', '조앤', 'Joanne', '류메잌미', 'RYU MAKE ME', '류꾸러기', '장꾸', '갓류진', '빛류진', '붐바야 소녀 신류진', '르냥이', '어우르', '신예술', '간지인간', '류순이', '삼똥', '만두', '걸크러쉬', '아리', '쾌녀', '똑똑이', '진주대장', '여신류진', '신류진경위', '호랑이베이비', '신다잘', '류돌프', '콩알깅', '류준표', '잔뜩 화난 고앵이', '핫걸', '여돌 콜렉터', '아기탕후루', '료진'],
        '채령': ['채령', '이채령', '령채', '솜털', '웃어? 걔', '령작가', '약사', '주디', 'Judy', '개복챌', '큐티챌', '순수결정체', '교수님', '고체령', '인간 기상청', '우바', '사모님', '정수리 요정', '령잔디', '밤하늘의 별', '프링글스 걔', '셀레나', 'Selena', '령미호'],
        '유나': ['유나', '신유나', '신나유', '핫세', 'Hussey', '슈신', '인간 에리얼', '에너자이저', '인간 비타민', '퀸유나', '유나봉쓰', '막냉이', '막둥이', '바비', 'MC가리비', '인간 있지', '뱅뱅이', '믿지나라 공주', '투리구슬']
    },
    'NewJeans': {
        '민지': ['민지', '김민지', '곰아지', '곰돌이', '백구', '뭉뭉비아 핫세', '밍디', '김믕디', '늑대', '포챠코', '주먹밥쿵야', '진실의 미간', '강아지 뾱', '체대여신 김민지', '고구마', '대한민지', '민지누나', '곰주님', '킴민지', '곰무원', '민지킴', '월요요정', '곰돌밍', '부시밍', '곰깅이', '김깻잎', '공식여신', '밍지', '밍구'],
        '하니': ['하니', '팜', '응옥', '범옥흔', '팜하니', '빵하니', '뿅오리', '빠마니', '아기토끼', '팜토끼', '파마니', '팜투리', '응옥이', '옥흔이', '팜팜언니', '팜팜팜', '감자', '팜후드', '팜자후', '코코팜', '포도알', '동그라미', '폼하니', '팜 미쳤다', '판다팜', '물개팜', '하니비', '팜마리오', '팜자님', '수달팜', '팜호초', '팜다이크'],
        '다니엘': ['다니엘', '마쉬', '모지혜', '다니', '모다니', '도다니', 'DoeDani', '모졔', '해바라기', '다니벨', '다니엘 마쉬멜로우', '뿌까니엘', '코다니엘', '다니에리얼', '다니엘 마쉬룸', '모마준', '녤자님', '모또당'],
        '해린': ['해린', '강해린', '강해린 이상하다', '아기고양이', '꼬양이', '냥냥', '우당이', '마름모 아티스트', '강고양이', '강고', '강떤고양이', '강떤여자', '언니조아고양이', '가트몬', '개구리', '침착걸', '강윤숙', '냥해린', '키티강', 'Kitty Kang', '부시강', '아기구미호', '강강강', '공식캣', 'Ms.river', '해린강', '바네사 강', '카피캣', '냥자님', '레몬제스트'],
        '혜인': ['혜인', '이혜인', '혜인', '데덴네', '갓기모델', '화보천재', '혜인스터', '탕탕이', '혜인깅이', '인간 어텐션', '아기 히데코', '장화신은 고양이', '도입부 장인', '최강막내', '혜인랑이', '쿼카', '혜니', '수도꼭지', '혜인 어린이', '아기 한나', '길쭉이', '혜인찌', '사춘기 짱구', '공식아가', '인간 Y2K', '혜인자님']
    },
    'aespa' : {
        '카리나': ['카리나', '유지민', '지민', '찌만', '디민', '지만', '찌민 등', '고양이', '카냥이', '쿠로미', '카로미', '메추리알', '메추리나', '족제비', '도토리', '대장', '리나', '망고스틴', '먹짱', '메텔', '병아리콩', '대장꼬부기', '아기뱜', '애기구미호', '월요요정', '유가네 막내딸', '유다정', '유알토', '유자', '유자몬', '유체리', '젭냥이', '짐만보', '징밍잉', '카깅이', '카랑둥', '카리더', '카수종', '카복치', '태봉산 날다람쥐', '토미에', '팔척 베이비', '하짱얼슬', '유짱구', '가리나', 'AI보다 더 AI', 'CG보다 더 CG', '신', 'stupid cheese cat', '김더스티', '잔퀸신종', '날카리'],
        '지젤': ['지젤', '우치나가', '애리', '김애리', '댓츠핫', '지구젤리', '우지젤', '힙합 댕댕이', '젤깅', '핫깅', '핫걸', '젤이', '제리', '지존', '더머', 'GiGi', '김애리(Kim Aeri)', '김카페사장', '앨이', '거북이', '김바다로 돌아가자', '김애리쓰기', '쩰', '가젤', '영애', '나애리', '기젯', '쿼카', '김혜정', '우치나파트라', 'GYM젤', '기젤레', '에리카', '지젤력', '생선바늘'],
        '윈터': ['윈터', '김민정', '민정', '윈터공주', '유영진이 성대로 낳은 딸', '민둥이', '겨울이', '먹선생', '김꿔바로우', '김동동', '김돼지고기 가마솥', '김민경훈', '김민둥맨둥', '김두부', '찐똥티엔', '김윈터', '김오또케', 'น้องหนาว', '대관령', '윤태', '댕정이', '덤', '거북이', '말티즈', '메론빵', '민툐링', '톨톨이', '비장한 무민', '맘단무', '빈터', '빈터르', '빈테르', '아기물만두', '아기백구', '얌얌윈터', '예삐', '용맹 백구', '윈깅', '윈또삐', '윈또체', '윈마멜', '윈테르', '유리잔 속 흰 강아지', '쩡이', '추운 친구', '토시오', '冬子', 'IM윈터', '감자', '양산 소녀', '김뽀'],
        '닝닝': ['닝닝', '닝이줘', '닝', '닝깅이', '닝이', '익익', '닝딩동', '닝이줰', '막닝이', '닝냥이', '아기 호랑이', '만두', '닝구', '닝왕', '닝킹', '닝치코치', '닝쌤', '닝이자식', '닝 안에 호랑이', '하얼빈 호랑이', '하얼빈 상여자', '에리얼', '닝부진', '닝꾸러기', '리틀 바다', '닝굴이', '닌닌', '닝삼줘', '밍']
    },
    'LE SSERAFIM': {
        '김채원': ['김채원', '채원', '쌈무', '채채', '아기 치타', '챔무', '김평', '코코볼', '쌈무요정', '쌈아치', '쌈코볼', '덤보', '쌈리더', '김포', '채노카', '코알라', '챙', '천사소녀', '쪼렙곰돌이', '자몽요정', '대천사', '무순', '러블리더', '채윈', '쌈채원', '김잡채원', '죄고의 언니', '밥의 아이', '김푸푸', '김풍푸', '음색요정', '깜빡공주', '김포키포키', '모닝채원', '김도독', '윤선비', '채소', '파워채채걸'],
        '사쿠라': ['사쿠라', '미야와키', '꾸라', '사쿠쨩', '체리 미야와키', '샄', '김쿠라', '쿠라', '미야와키상', '걸꾸라시', '커비', '샄냥이', '김고양이', '아천고', '벚꽃', '김부각', '미야와키 프로', '사쿠라테스', '날꾸람쥐', '사쿠로미', '근육고양이', '명언자판기', '김유나', '김윤아', '크루아상', '두부킬러', '치즈냥이', '꾸꾸', '사키', '샛바닥', '세프라'], # '고양이' 제거
        '허윤진': ['허윤진', '제니퍼', '젠', '핫걸', '허댕이', '기린쨩', '기린', '골댕이', '허박벌', '파괴왕', '젠또울', '젠또줄', '음색 여왕', '언어 천재', '미국 하이틴 드라마 여주인공', '팔척강아지', '물만두', '허트리버', '허란젓', '이유리', '천사 개', '히허로'],
        '카즈하': ['카즈하', '나카무라', '즈하', '무민', '가즈아', '하짱', '백조', '강주하', '선무꾼', '믕믕이', '두부', '순두부', '프로틴초고함량두부', '어남카', '카씨 가문 막내', '서가영', '오사카 개그우먼', '나뭇잎', '쥬하', '박연주', '카나지', '즈하노사우루스'],
        '홍은채': ['홍은채', '은채', '만채', '스마일 감자', '은채아기', '아기 사자', '포켓몬', '음체', '병아리', '우리 막내 은채', '아기 은행장', '라이옹', '잼민이', '홍바오', '쪽쪽이', '만채파워', '화요요정', '은체리', '훈채'], # '아기 고양이' 제거
        '김가람': ['김가람']
    },
    'IVE': {
        '안유진': ['안유진', '유진', '안댕댕', '댕댕이', '맑은 눈의 광인', '포포', '카뿜리', '윶', '안콩알', '양파쿵야', '안리더', '안리다', '댕리더', '댕리다', '스타쉽의 여왕', '안유댕', '큰댕', '안유딩', 'MC유진', '안둘리', '자동심장충격기', '안유델', '반장댕댕', '반유진', '탐지견', '대한의 보물', 'K-POP이 낳은 천사', '안유후진', '엔딩요정', '안유지니어스', '캐thㅣ', '나유진', '유쿠키', '안키치', '대전의 딸', '지구오락실', '밥진이', '안퀀스', '킥진이', '핫도그', '강안지'],
        '가을': ['가을', '김가을', '가을 선배', '가을선배', '가을양', '가을왕자님', '김피에르가을', '폴수종', '갤', '걀걀', '나무늘보', '대롬쥐', '맏내', '아키쨩. 秋ちゃん', '아키코', '秋子', '종이인형', '웃수저', '가을 센파이', '가으루', 'ガウル', '가으리', '으리', '왕도토리', '할머니', '그랜마', '포리', '폴끼', '꾸짖을 갈', '이누야샤', '폴냥', '폴키', '달이'],
        '레이': ['레이', '나오이', '콩순이', '콘수니', '김레이', '레요미', '키레이', '렝', '쁘유', '명이나물', '레이코', '레이푱', '찹쌀떡', '쿠로미', '팔척콩순이', '나가이 레이', '힙합콩순', '레찌', '흐린 눈의 광인', '정령', '잔디', '마루', '레이첼', '김영희', '카와이 레이', '나오리'],
        '장원영': ['장원영', '원영', '워뇨', '워녕', '102', '갓기', '국민 손녀', '공포리', '녕토끼', '아기토끼', '노벨 아이돌상', '망나뇽', '만년돌', '모태 센터', '밥파고', '밥무새', '워녕스프리', '이촌동 꿀주먹', '이촌동 한우 장인', '자이언트 베이비', '장비키', '장한우', '체리토끼', '콩원영', '원콩이', '효녀', '체리'],
        '리즈': ['리즈', '김지원', '지원', '치즈냥이', '치즈불닭냥이', '깜냥이', '베이지냥', '포도냥이', '누룽지냥이', '인절미냥이', '금발좌', '노란머리 걔', '김리즈', '리즈 시절', '재규어', '음색요정', '울보', '울보냥이', '뿌앵냥', '오열냥', '물만두', '토마토', '토마토냥이', '아기냥이', '곰도리즈', '곰냥이', '공주', '지워니', '아기 감귤', '맂깅', '바비인형', '왕감자', '명창 고양이', '좋아마시', '아공깜', '제주도 새우 장인', 'German Liz', '잠만보', '와사비', '당황이', '치즈'],
        '이서': ['이서', '이현서', '현서', '아기호랑이', '이서아기', '막냉이', '막냉이서', '잇깅', '잼민이서', '마이멜로디', '금쪽이', '금쪽이서', '갓기서', '멘트 자판기', '윙크 자판기', '메타몽', '호랑이서', '떠깅이', '이랑이'] #'애기', '갓기' 제거
    },
    'NMIXX': {
        '릴리': ['릴리', '머로우', '박진', 'LOML', '릴황', '릴언', '백합', '리리', '릴리아나 그란데', '릴욘세', '순수악', '릴뭉이', '레몬사탕', '릴모지', '릴리짱박머로우', '머라이어 캐릴리', '릴자님', '미담제조기', '뒈지지않는4세대최고의보컬', '비율천재', '아기명창웜뱃', '맏내', '바닐라', '박릴리', '리얼리릴리', '릴교수', '인간 각도기', '기계', '딸깍릴황'],
        '해원': ['해원', '오해원', '농담곰', '올라프', '밈천재', '밈해원', '또 오해원', '아기 백구', 'JYP 공채 개그맨 1기', '행담곰', '행토리', '오리더', '겨자', '겨울 여자', '쇼츠 여신', '왕눈이', '기린', '콩쥐', '오오', 'MC더덕', '백오옥', '오해단 감독', '93년생 육해원', '별의 커비', '오돌뼈', '오백원', '행자님', '바게트 복근 섹시 천사', '마그네틱 오', '오스틴', '바보뚜껑', '노래 자판기', '특전사', '미스코리아', '외모췍'], #'행' 제거
        '설윤': ['설윤', '설륜', '윤아', '아기 토끼', '아기 사슴', '보름달 걔', '구체 설윤', '사토', '아사토', '밤비', '설윤아씨', '설윤엥', '설만보', '떨유나', '마이멜로디', '프로 아이돌', '왕눈이', '윤아 공주', '설레임', '선녀', '설쁘', '신데렐라', '이뿌니 이뿌니', '슈퍼스타 설윤(슈스설)', '(설)장군님', '피구왕 통키', 'SNOW'],
        '배이': ['배이', '배진솔', '진솔', '솔이', '내 사랑 진솔 공주님', '오끼', '맑은 눈의 광인', '배양자', '배화백', '배딘돌', '배젤', '반짝거리는 눈의 광인', '텅 빈 눈의 광인', '엔믹스의 찐광기', '기영이', '배지터', '배이비', '배럭지', '배발', '배꺼비', '연말배이', '배아 리파', '배춘솔', '파괴요정', '배입부', '양산 마취총', '섹시 다이너마이트', '배꾸라지', '배진흙 선생님'],
        '지우': ['지우', '김지우', '쮸', '김쮸', '쥬', '마이쮸', '옆태장인', '쮸쪽이', '금쮸기', '금쪽이', '말랑쥬', '말티쥬', '강아지우', '체리 지우', '포차코', '쮸차코', '팥쥐', '응지우', '지우집중', '두콩이', '미스코리아'],
        '규진': ['규진', '장규진', '뀨', '귲', '규냥이', '진규', '짱규진', '앙고', '대갓기', '애깅이', '분당아기고구마', '삼김 고양이', '쿠로미', '인간 귤', '디.번.포', '진규장', '앙고고'],
        '지니': ['지니', '최윤진', '윤진', '호랭이', '호랭 지니', '최지니']
    },
    'tripleS': {
        '윤서연': ['윤서연', '서연', '근본', '포차코', '햄햄', '멋쟁이', '이성 소녀', '이과 소녀', '대전 소녀', '엄마', '떠연', '단군', '럭키햄찌', '럭키햄햄', '햄디'],
        '정혜린': ['정혜린', '혜린', '리네', '아깽이', '아기 천재', '셰프', '정아가', '애기린', '혀린', '정혜리더', '혜리더', '정리더', '혜밤'],
        '이지우': ['이지우', '지우', '쥬', '지우개', '아가지우', '티저여신', '아이스크림 걔', '하드우', '잠만쥬', '쥬만보', '냠냠지우', '돌멩이', '주먹밥쿵야', '쥬디'],
        '김채연': ['김채연', '채연', '챈', '채욤미', '어피치', '채욘', '째욘', '벚채', '채연하니', '씃채', '코기뮹', '이슬이누나', '김채워연'],
        '김유연': ['김유연', '유연', '이대여신', '유연아씨', '춘식이', '김뻣뻣', '안유연', '인간산리오', '김북극곰', '곰유연', '유연웅니', '연유'],
        '김수민': ['김수민', '수민', '뚜마니', '쑤마닝', '뚜마닝', '수바타', '내가 만든 포트홀', '애깅', '달팽이', '비밀소녀', '다람쥐'],
        '김나경': ['김나경', '나경', '나키', 'Naky', '도도한 꽃사슴', '나냥', '삐동', '안무단장', '나경의 세계', '김나갱', '중영'],
        '공유빈': ['공유빈', '유빈', '공유밤', '유밤', '꽁유밤', '하트머리 걔', '아기호랑이', '육수의 여왕', '공주빈', '중도포기'],
        '카에데': ['카에데', '야마다', '카에', '에데', '단풍이', '단풍여자', '카에짱', 'かえちゃん', '카에뿅', 'かえぴょん', '공카에', '까에'],
        '서다현': ['서다현', '다현', '소다', '체리소다', '토깽이', '서다횬', '황금보컬', '음색여왕', '독.팽.광', '독팽광'],
        '코토네': ['코토네', '카미모토', '토네', 'とね', 'トネ', '코탸', 'こてゃ', '코짱', '네네', '인간 지브리', '박토네', '명예 한국인', 'GOAT토네', '내 분량 어디갔토네', '코토콜라'],
        '곽연지': ['곽연지', '연지', '곽곽곽연지', '휴지곽', '곽부호', '초코쿠키', '초월읍 미소녀', '김끼루룩끼루룩', '샤인 베러'],
        '니엔': ['니엔', '쉬니엔츠', '니엠', 'Nancy', '허니엔', '외국인 김수민', '녠', '넨름이'],
        '박소현': ['박소현', '소현', '박쌤', '보노보노', '흐물이', '박소현의 러브게임', '소효니', '홋홋이'],
        '신위': ['신위', '저우신위', '여신위', 'Felicia', '저우지에', '곽신위', '중국 천송이', '시니', '불륜의 여신', '배신위', '밥신위'],
        '마유': ['마유', '코우마', '막내', '코우마쨩', '딸기 솜사탕', '딸기쿠키', '코구마유'],
        '린': ['린', '카와카미', '카와린', '샤메린', '리니', '린냥이', '린바이 병장', '키끄 린'],
        '주빈': ['주빈', '주비니', '보스 베이빈', '비니', '콩이', 'Jasmine', '아기 공주빈', '주밤', '주나운서'],
        '정하연': ['정하연', '하연', '다람쥐', '햄스터', '코코볼', '뽀뽀 요정', '가을 여자', '육각단 대장', '정화백', '치워리더', '반말핑'], # '고양이', '고슴도치', '족제비', '뱀' 제거
        '박시온': ['박시온', '시온', '박성변조', '빵시온', '먹잘알', '애기목소리', '참새', '인간 논스케일'],
        '김채원': ['김채원', '채원', '꼬부기', '딸기 공주', '부부의 세계'],
        '설린': ['설린', '피라다', '분락사', '말랑설린', '타이 프린세스', '곽설린', '박설린', '김설린'],
        '서아': ['서아', '정해린', '해린', '정서아', '서아가', '하린', '써니', '떠아룽바오'],
        '지연': ['지연', '지서연', '연이', '도준', '백조', '서연리나', '김지연', '믈리']
    },
    'FIFTY FIFTY': {
        '키나': ['키나', '송유니', '경자', '송키나'],
        '예원': ['예원', '손예원', '아기토끼', '손캡틴'],
        '문샤넬': ['문샤넬', '샤넬', '샤샤', '문샤샤', '문치타', '미어캣', '샤돌이', '샤넬 벨', '넬샤'],
        '하나': ['하나', '임하람', '하람', '하나님', '개죽이', '나무늘보', '취미부자', '비율깡패', '뮤지돌'],
        '아테나': ['아테나', '아테나 양', '뱁새', '테나', '자꺄', '안테나'],
        '새나': ['새나', '정세현', '춤 추는 알파카', '정새나'],
        '아란': ['아란', '정은아', '쿼카', '아기백구', '루피', '할미넴', '정아란'],
        '시오': ['시오', '정지호', '호지', '정시오'],

    },
    'STAYC': {
        '수민': ['수민', '배수민', '숨', '배숨', '토냥이', '숨깅', '배리더', '숨도서관', '숨니', '쑴니', '까치발숨', '배토끼', '자스민', '자수민', '말실수민', '숨몬스', '뚜미', '배넬로피', '돌콩', '뿌니', '리본수민', '거민이', '숨디', '꽈당수민', '수미니', '배숨니', '아기고슴도치', '팩폭쑴', '대장토끼', '아기토끼', '콩알', '아기체리쥬빌레', '포도사탕', '상도', '까꿍이'],
        '시은': ['시은', '박시은', '씬', '씬니', '시니', '박시금', '말랑공주', '순두부말랑이', '꿀짱구', '굴짬뽕', '씬짱', '진실의 귀', '은시', '쫄보시은', '씬푼젤', '아기 마시멜로', '아니야나 동그란데', '씬또몰', '깐달걀', '박유애나시은', '박베레모시은', '박비빔밥시은', '박뭐든가능시은', '씬나모롤', '윤시은', '알밤도토리', '밤의 요정', '빡시', '시은스쿨', '박감자', '햄토리'],
        '아이사': ['아이사', '이채영', '채영', '아이사랑해', '챙', '챙니', '챙냥이', '아이사장님', '아의사', '이사님', '아쏴', '말랑 고양이', '아이셔', '스테이씨의 엄마', '아이삼', '의사몽쥴', '아기 챙깨비', '챙이', '얼음공주', '모찌이사', '돌멩이사', '이채소', '힙챙', '아무사', '고양이공주'],
        '세은': ['세은', '윤세은', '센', '센니', '복뚱이', '진실의 콧구멍', '스포요정', '센또삐', '셍', '센깅', '윤센', '말랑 복숭아', '왕여신', '백설공주', '인간 명화', '인간 유화', '분노의 콧구멍', '북극여우', '세바스찬', '작은 공주', '아기 여우', '체리', '셍니', '얼굴 천재', '회콩센', '어쩔세은', '꼬부기', '초콜릿 요정', '빛세은', '스테이씨 공식 아가', '멋있는 여자', '굥세긍', '잘이없딱', '셀 공주', '쩝쩝세은', '스포배신요정', '공주세은', '윤세금', '세은이', '윤대리', '세우소', '포주친 미인구', '센냥이'],
        '윤': ['윤', '심자윤', '자윤', '아기 호랑이', '윤자', '인간 심즈', '심즈윤', '룰러윤', '심엔프피자윤', '윤좌', '굥', '풀뱅 걔', '바비인형', '뽀뽀귀신', '심자윤', '심윤자', '베이비', '자유닝', '심짱구', '호랭이', '재이 언니', '시은 엄마', '포키', '자굥', '심바', '심네버랜드자윤', '윤디'],
        '재이': ['재이', '장예은', '예은', '짱재이', '장재이', '짱티쥬', '말랑말티쥬', '아기 도깨비', '단발 걔', '꽃사슴', '존예은', '너는 왜 우뤄', '윌리엄', '용맹말티쥬', '짱예', '장뻔뻔', '재이공주', '이재', '재이콜', '짱재이', '장차차', '쟁', '째이', '짱예은', '연말재이', '월요정', '재이재이', '장금쪽', '🐶장티쥬🐶', '예으닝❤', '말공이', '예으닝', '맑눈광'],
    },
    'Kep1er': {
        '최유진': ['최유진', '유진', '유토끼', '최토끼', '대장토끼', '유리더', '햄토리', '동공지진녀', '페이크 막내', '군필 아이돌', '이쑤시개 팔', '사과 같은 얼굴', '팝리콘', '최유연', '쿠로미', '지니', '산독기', '리어루바비', '성수동바비'],
        '히카루': ['히카루', '에자키', '카루', '빛카루', '히카룽', '잼카루', '히카쨩', '히짱', '먹카루', '에자키 뻔뻔', '히카부', '히카츄', '에자키 김가루', '심쏘', '소정 언니', '팅', '팅이', '팅팅', '팅냥이', '션샤우팅', '냅다 샤우팅', '션데렐라', '션샤인', '유니콘', '천샤오팅', '꽃사슴'],
        '샤오팅': ['샤오팅', '션샤오팅', '심소정', '심쏘', '소정 언니', '팅', '팅이', '팅팅', '팅냥이', '션샤우팅', '냅다 샤우팅', '션데렐라', '션샤인', '유니콘', '천샤오팅', '꽃사슴'],
        '휴닝바이에': ['휴닝바이에', '휴닝바히에', '바히에', '휴닝바히', '잘레', '휴닝', '정바히에', '히에', '아기사자', '아기밤비', '휴바', '열정파이리', '에깅이', '힘쎈여자 닝남순', '4th Gen Princess', '금발 여신'],
        '김채현': ['김채현', '채현', '채로미', '음색 요정', '음색요정', '하트 강쥐', '하트강쥐', '천사 목소리', '천사목소리', '아기 명창', '아기명창', '명창 토끼', '명창토끼', '명창 아기 토끼', '명창아기토끼', '비주얼 메보', '비주얼메보', '캐릭터 만렙', '캐릭터만렙', '말랑콩떡', '말랑 토끼', '말랑토끼', '하얀 말랑 토끼', '하얀말랑토끼', '채토끼', '김토백', '백호토끼', '토외백카', '산독기', '백설공주', '채채', '챙구', '채장금', '인생네컷 장인', '인생네컷장인', '네컷 장인', '네컷장인', '얼굴 장인', '얼굴장인', '포카 장인', '포카장인', '셀카 장인', '셀카장인', '눈의 여왕', '눈의여왕', '인간 찹쌀떡', '인간찹쌀떡', '인간 백설기', '인간백설기', '인간 순두부', '인간순두부', '아기 토끼', '아기토끼', '아기 백호', '아기백호', '아기 백곰', '아기백곰', '아기 천사', '아기천사', '채현 공주', '채현공주', '채혀니', '현채', '채랑이', '스포 요정', '스포요정', '복숭아 주스', '복숭아주스', '금손', '펀치 여왕', '펀치여왕', '현이', '혀니', '채니', '집순이', '방순이', '침대순이', '거울 공주', '거울공주'],
        '서영은': ['서영은', '영은', '사막여우', '아기여우', '영근이', '앉아영은', '댄스 맛집', '댄스맛집', '영웅', '서스윗', '사막여우양', '영은몬', '완사그', '똘망똘망 사막여우', '똘망똘망사막여우', '영크아이', '영으니', '여은이', '영응', '영데렐라', '떠영은', '천방지축 떠영은', '천방지축떠영은', '윙크 요정', '윙크요정', '셩은'],
        '김다연': ['김다연', '다연', '뎐람쥐', '퀸다연', '도입부 장인', '도입부장인', '댄싱퀸', '댄스 요정', '댄스요정', '콩순이', '콩다연', '대다연', 'D연', '댄스 부장', '댄스부장', '다연쌤', '요니', '밥'],
        '마시로': ['마시로', '사카모토', '시로', '마시멜로', '마셰프', '마장금', '마시마로', '무백', '너구리', '귤시로', '500', '폰예서', '노랑이', '카리스마시로'],
        '강예서': ['강예서', '예서', '강토끼', '아기토끼', '막내토끼', '강초딩', '말 많은 애기', '말많은애기', '강냉이', '깡', '거울 공주', '거울공주', '마지막 예서', '마지막예서', '진짜 예서', '진짜예서', '예카소']
    }
}

In [85]:
def extract_keywords_frequency(series_word_list, top_n=30, min_df=1, existing_keywords=None):
    # 시작시간 확인
    start_time = time.time()

    # 기존 키워드가 없으면 빈 리스트로 초기화
    if existing_keywords is None:
        existing_keywords = []

    # 단어별 문서 빈도를 저장할 딕셔너리 (각 단어가 등장한 문서 수)
    doc_frequency = defaultdict(int)

    # 각 문서에서 등장한 단어 추출 및 빈도 계산
    for word_list in series_word_list:
        unique_words_in_doc = list(set(word_list))  # 문서 내에서 중복된 단어는 한 번만 카운트
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # min_df 조건을 충족하는 단어들의 빈도를 저장할 딕셔너리
    word_frequencies = defaultdict(int)

    # 각 문서의 단어 빈도 카운트 (min_df 조건에 맞는 단어만 빈도 계산)
    for word_list in series_word_list:
        for word in word_list:
            if doc_frequency[word] >= min_df:
                word_frequencies[word] += 1

    # 상위 빈도 단어 추출
    sorted_words = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)

    # 기존 키워드 제외하고 새로운 키워드만 추출
    new_keywords_with_weights = [(word, count) for word, count in sorted_words if word not in existing_keywords]
    
    # 상위 n개 키워드만 추출 (중복 제외 후)
    top_keywords_with_weights = new_keywords_with_weights[:top_n]

    # 키워드만 추출
    top_keywords = [word for word, _ in top_keywords_with_weights]

    # 가중치를 전체 합으로 표준화
    total_count = sum(count for _, count in top_keywords_with_weights)
    if total_count > 0:
        keywords_with_weights = [(word, count / total_count) for word, count in top_keywords_with_weights]
    else:
        keywords_with_weights = [(word, 0) for word, count in top_keywords_with_weights]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [86]:
def extract_keywords_tfidf(series_word_list, top_n=100, min_df=1, existing_keywords=None):
    import time
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer

    # 시작시간 확인
    start_time = time.time()

    # 기존 키워드가 없으면 빈 리스트로 초기화
    if existing_keywords is None:
        existing_keywords = []

    # 리스트 시리즈를 문자열 시리즈로 변형 (빈 리스트는 빈 문자열로 처리)
    word_list_str = series_word_list.apply(lambda words: ' '.join(words) if len(words) > 0 else '')

    # 빈 문자열이 있는지 확인하고 제거
    word_list_str = word_list_str[word_list_str.str.strip() != '']

    # word_list_str이 비어 있는지 확인
    if word_list_str.empty:
        print("Empty document list, skipping TF-IDF extraction.")
        return [], []

    # TF-IDF 벡터화 (stop_words=None으로 불용어 제거 비활성화)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=min_df, stop_words=None)
    tfidf_matrix = vectorizer.fit_transform(word_list_str)

    # 각 단어의 TF-IDF 점수를 추출
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=0)  # 전체 문서에서 평균 TF-IDF 추출

    # TF-IDF 점수를 기준으로 상위 키워드 추출
    top_n_indices = tfidf_scores.argsort()[::-1]  # 높은 TF-IDF 순으로 정렬
    top_keywords = [feature_names[idx] for idx in top_n_indices if feature_names[idx] not in existing_keywords]

    # 상위 n개 키워드만 추출
    top_keywords = top_keywords[:top_n]

    # 각 단어의 TF-IDF 점수 추출
    top_n_indices = [i for i, word in enumerate(feature_names) if word in top_keywords]
    weights = [tfidf_scores[idx] for idx in top_n_indices]

    # 가중치를 전체 합으로 표준화
    weights = np.array(weights) / np.sum(weights)

    # 키워드와 가중치를 묶어서 반환
    keywords_with_weights = list(zip(top_keywords, weights))

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights


In [87]:
import itertools

# 키워드 리스트 초기화    
keyword_list = pd.DataFrame(columns=['Group', 'Title', 'Member', 'keyword'])

# 그룹별 멤버와 인식어 리스트를 정의합니다.
# nicknames = { ... }

# Group, Title 열에 속한 그룹들의 리스트
groups = sample_df['Group'].unique()
titles = sample_df['Title'].unique()

# 각 그룹별 멤버와 전체에 대한 키워드 추출
for group in groups:
    if group in group_members:  # group_members에 있는 그룹만 처리
        for member in group_members[group]:
            group_df = sample_df[sample_df['Group'] == group]
            mem_idx = group_df['members'].apply(lambda x: member in x if isinstance(x, list) else False)
            word_list = group_df[mem_idx]['word_list']
            keywords, weights = extract_keywords_tfidf(word_list, top_n=30, min_df=1)

            # 해당 멤버의 인식어는 배제하지 않고, 그룹 내 다른 멤버의 인식어만 배제
            #other_member_aliases = list(itertools.chain.from_iterable(
            #    alias for mem, alias in nicknames[group].items() if mem != member
            #))

            # 모든 인식어 제외
            member_aliases = list(itertools.chain.from_iterable(
                alias for mem, alias in nicknames[group].items()))

            # 다른 멤버 인식어 제외
            filtered_keywords = [kw for kw in keywords if kw not in member_aliases]
            filtered_weights = [weights[i] for i, kw in enumerate(keywords) if kw not in member_aliases]  # 가중치도 필터링

            # 새로운 키워드를 기존 리스트에 추가
            frac = pd.DataFrame()
            frac['keyword'] = filtered_keywords
            frac['weights'] = filtered_weights  # 필터링된 키워드만큼의 가중치 할당
            frac['Group'] = group
            frac['Title'] = 'All'
            frac['Member'] = member

            frac = frac[:5]    
            keyword_list = pd.concat([keyword_list, frac])
            print(group, member, len(keywords))

# MV별로 새로운 키워드 추출
for title in titles:
    title_df = sample_df[sample_df['Title'] == title]
    group = title_df['Group'].iloc[0]

    if group in group_members:  # group_members에 있는 그룹만 처리
        for member in group_members[group]:
            mem_idx = title_df['members'].apply(lambda x: member in x if isinstance(x, list) else False)
            word_list = title_df[mem_idx]['word_list']
            keywords, weights = extract_keywords_tfidf(word_list, top_n=30, min_df=1)

            # 모든 인식어 제외
            member_aliases = list(itertools.chain.from_iterable(
                alias for mem, alias in nicknames[group].items()))

            # 인식어 제외
            filtered_keywords = [kw for kw in keywords if kw not in member_aliases]
            filtered_weights = [weights[i] for i, kw in enumerate(keywords) if kw not in member_aliases]  # 가중치도 필터링

            # 새로운 키워드를 기존 리스트에 추가
            frac = pd.DataFrame()
            frac['keyword'] = filtered_keywords
            frac['weights'] = filtered_weights  # 필터링된 키워드만큼의 가중치 할당
            frac['Group'] = group
            frac['Title'] = title
            frac['Member'] = member
            
            # 상위 5개 결과로 제한
            frac = frac[:5]
            keyword_list = pd.concat([keyword_list, frac])

# 인덱스 초기화 및 csv 익스포트
keyword_list = keyword_list.reset_index(drop=True)
keyword_list.to_csv('member_keyword_list_top5.csv', encoding='utf-8-sig', index=False)

728개 댓글 키워드 추출에 걸린 시간: 0.016407012939453125 초

NMIXX 릴리 30
551개 댓글 키워드 추출에 걸린 시간: 0.004354000091552734 초

NMIXX 해원 30
630개 댓글 키워드 추출에 걸린 시간: 0.004846096038818359 초

NMIXX 설윤 30
376개 댓글 키워드 추출에 걸린 시간: 0.0033180713653564453 초

NMIXX 배이 30
577개 댓글 키워드 추출에 걸린 시간: 0.004263162612915039 초

NMIXX 지우 30
517개 댓글 키워드 추출에 걸린 시간: 0.0034821033477783203 초

NMIXX 규진 30
257개 댓글 키워드 추출에 걸린 시간: 0.003367900848388672 초

NMIXX 지니 30
1722개 댓글 키워드 추출에 걸린 시간: 0.00985407829284668 초

ITZY 예지 30
1632개 댓글 키워드 추출에 걸린 시간: 0.009037971496582031 초

ITZY 리아 30
3831개 댓글 키워드 추출에 걸린 시간: 0.022286176681518555 초

ITZY 류진 30
2162개 댓글 키워드 추출에 걸린 시간: 0.010790109634399414 초

ITZY 채령 30
1494개 댓글 키워드 추출에 걸린 시간: 0.007313728332519531 초

ITZY 유나 30
5000개 댓글 키워드 추출에 걸린 시간: 0.09018611907958984 초

NewJeans 민지 30
8002개 댓글 키워드 추출에 걸린 시간: 0.12817001342773438 초

NewJeans 하니 30
7570개 댓글 키워드 추출에 걸린 시간: 0.14790701866149902 초

NewJeans 다니엘 30
2629개 댓글 키워드 추출에 걸린 시간: 0.023019075393676758 초

NewJeans 해린 30
5994개 댓글 키워드 추출에 걸린 시간: 0.114553213119506

In [88]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

#keyword_list
keyword_list['Group'].value_counts()
keyword_list.to_csv('member_keyword_list_top5_tfidf.csv', encoding='utf-8-sig', index=False)

In [100]:
# 'Group'과 'Title'로 그룹화한 후, Keyword를 와이드 포맷으로 변환
keyword_list['Keyword_Index'] = keyword_list.groupby(['Group', 'Title', 'Member']).cumcount() + 1

# 'Keyword_Index'를 열로 피봇하여 와이드 포맷으로 변환
df_wide_format = keyword_list.pivot(index=['Group', 'Title', 'Member'], columns='Keyword_Index', values='keyword')

# 결과 출력
#df_wide_format.to_csv('member_keyword_list_top5_wide_format.csv', encoding='utf-8-sig', index=False)
df_wide_format

Keyword_Index                                                         1  \
Group       Title                                         Member          
FIFTY FIFTY All                                           문샤넬       피프티   
                                                          새나         노래   
                                                          시오         노래   
                                                          아란         음색   
                                                          아테나       피프티   
                                                          예원        피프티   
                                                          키나         노래   
                                                          하나         노래   
            Cupid                                         문샤넬        디올   
                                                          새나         생일   
                                                          시오         노래   
                                                          아란        목소리   
                                                          아테나       목소리   
                                                          예원        큐피드   
                                                          키나        피프티   
                                                          하나         노래   
            Higher                                        문샤넬        여서   
                                                          새나         노래   
                                                          시오         보컬   
                                                          아란         음색   
                                                          아테나        출발   
                                                          예원         출발   
                                                          키나         노래   
                                                          하나         노래   
            Lovin' Me                                     문샤넬        프로   
                                                          새나         노래   
                                                          시오         노래   
                                                          아란         텔미   
                                                          키나        화이팅   
                                                          하나         노래   
            SOS                                           문샤넬       피프티   
                                                          새나       계약해지   
                                                          시오         삼촌   
                                                          아란         음색   
                                                          아테나       피프티   
                                                          예원        피프티   
                                                          키나         노래   
                                                          하나         노래   
            Starry Night                                  문샤넬       피프티   
                                                          새나         무시   
                                                          시오         노래   
                                                          아란         음색   
                                                          아테나       피프티   
                                                          예원        피프티   
                                                          키나        피프티   
                                                          하나         생일   
ITZY        All                                           류진         노래   
                                                          리아         노래   
                                                          예지        사랑해   
                                                          유나        사랑해   
                                                          채령         파트   
            BET ON ME    

In [90]:
sample_df.head()

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list,members
0,NMIXX,별별별 (See that),node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,True,"[중독, 진심, 노래, 의문, 엔믹스]",NaN
1,NMIXX,별별별 (See that),node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,True,"[기도, 아쉬움, 대쉬, 금도, 대중성, 노래]",NaN
2,NMIXX,별별별 (See that),node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,True,"[본부장, 엘범, 천만, 클릭, 연락, 연결, 취해, 피씨, 뮤비, 노래, 뷰수, ...",NaN
3,NMIXX,별별별 (See that),node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,True,[노래],NaN
4,NMIXX,별별별 (See that),node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,True,"[걸그룹, 멤버, 빵빵, 노력, 묵자, 얼마나, 세대, 외모췍, 더더, 리더, 해원]",[해원]


In [91]:
#sample_df[
#    sample_df['word_list'].apply(lambda x: '어서' in x if isinstance(x, list) else False)
#]

In [92]:

result = sample_df[
    (sample_df['Group'] == 'NewJeans') & 
    (sample_df['Title'] == 'How Sweet')&
    (sample_df['members'].apply(lambda x: '하니' in x if isinstance(x, list) else False))
]
extract_keywords_tfidf(result['word_list'], top_n=30, min_df=1, existing_keywords=None)

640개 댓글 키워드 추출에 걸린 시간: 0.007474184036254883 초



(['하니',
  '뉴진스',
  '민지',
  '단발',
  '혜인',
  '해린',
  '다니엘',
  '버니즈',
  '뮤비',
  '하이브',
  '민희진',
  '노래',
  '사랑해',
  '머리',
  '응원',
  '어도어',
  '활동',
  '무대',
  '방시혁',
  '제한',
  '단월드',
  '고양이',
  '화이팅',
  '사이비',
  '돼지',
  '프로',
  '시간',
  '멤버',
  '표현',
  '장면'],
 [('하니', 0.015709985213175653),
  ('뉴진스', 0.02814143706382267),
  ('민지', 0.06444170880985146),
  ('단발', 0.043430301799916735),
  ('혜인', 0.0563532787056893),
  ('해린', 0.016265204421647417),
  ('다니엘', 0.014686022017352604),
  ('버니즈', 0.023198231736168615),
  ('뮤비', 0.01394010018468321),
  ('하이브', 0.01659709771653099),
  ('민희진', 0.03449293746214398),
  ('노래', 0.05721485855877777),
  ('사랑해', 0.029114576698583347),
  ('머리', 0.016505392527025143),
  ('응원', 0.03673958539143336),
  ('어도어', 0.026320226905979346),
  ('활동', 0.015390570507162117),
  ('무대', 0.014190455464876395),
  ('방시혁', 0.017049647638065362),
  ('제한', 0.02063532132254474),
  ('단월드', 0.013880677975624223),
  ('고양이', 0.016347769766504285),
  ('화이팅', 0.013935140573555233),
  ('사이비', 

In [93]:
# 특정 조건으로 행 찾기 (group, title, member)
sample_df[
    (sample_df['Group'] == 'NewJeans') & 
    (sample_df['Title'] == 'How Sweet')&
    (sample_df['members'].apply(lambda x: '하니' in x if isinstance(x, list) else False))
][:100]

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list,members
280673,NewJeans,How Sweet,node,Ugy-se1Lam5NBppCA3R4AaABAg,NaN,시간은 뮤비 기준 [timecode] 처음에 해린이 영상 통화하는 장면 속 캐릭터 ...,@뉴진스사랑해-w1x,2024-10-01T15:39:52Z,2,True,"[오리, 현실, 단월드, 계정, 입성, 가슴, 뮤직비디오, 오토바이, 기준, 취향,...","[민지, 하니, 다니엘, 해린, 혜인]"
280675,NewJeans,How Sweet,node,Ugx7VWuh3eJ37bqK-LB4AaABAg,NaN,백날 막아봐라 조회수는 계속 오를거다 뉴진스의 모든 곡이 그러하듯 슴슴하니 중독적인...,@반도의셀레네이터-k5k,2024-10-01T05:24:16Z,9,True,"[김치, 루틴, 뉴진스, 사고, 중독, 음원, 국물, 슴슴하, 일상, 뮤비, 조회수...",[하니]
280733,NewJeans,How Sweet,node,UgwTSlRwhICAnqNPIfp4AaABAg,NaN,하니.씨 . 제가.정말.사랑합니다.,@Skdjfnghvhfhdbeufhabdsiweurfh,2024-09-20T05:32:31Z,4,True,[하니],[하니]
280739,NewJeans,How Sweet,node,UgyROGwXpyfhjXoxkSN4AaABAg,NaN,힙하니,@GOODSPEED-c8v,2024-09-16T23:51:16Z,5,True,[힙하],[하니]
280740,NewJeans,How Sweet,node,UgyKlvCRRCn50XNBsZx4AaABAg,NaN,근데 진짜 뮤비도 그렇고 스타일링도 그렇고… 너무 이쁘고 참신하네 역시 뉴진스와 민...,@user-vn5uu1sw6e,2024-09-16T17:14:17Z,14,True,"[뉴진스, 일링, 응원, 민희진, 뮤비, 스타]",[하니]
280742,NewJeans,How Sweet,node,Ugyse4pT59d8KWjfmm14AaABAg,NaN,하니는 귀엽고 멋있다 그리고 혜인이가 다리 다쳐서 안 나와서 너무 아쉽다. 뉴진스 ...,@변찬미-n2n,2024-09-16T07:54:33Z,4,True,"[뉴진스, 하니, 혜인, 다리, 사랑해]","[하니, 혜인]"
280816,NewJeans,How Sweet,node,Ugy_ynWOym-HOT4Csg14AaABAg,NaN,여기 하니 진짜 미침,@뚜뚜-u8q,2024-09-10T08:03:46Z,4,True,[하니],[하니]
280875,NewJeans,How Sweet,node,Ugxu6qNwOORw2CfiBJ94AaABAg,NaN,[timecode] 처음에 해린이 영상 통화하는 장면 속 캐릭터 돼지(=방)와 곰(...,@6hhvvji,2024-09-03T02:52:16Z,13,True,"[오리, 현실, 단월드, 계정, 입성, 가슴, 뮤직비디오, 오토바이, 전달, 취향,...","[민지, 하니, 다니엘, 해린, 혜인]"
280893,NewJeans,How Sweet,reply,NaN,UgxRn-juv13C4CEoCo54AaABAg,​@@June-cm5qb 그렇네요. 뉴진스를 하이브화할 모양이네요. 그래도 애들이 ...,@User7hugfrji-o4d,2024-09-02T11:31:07Z,3,True,"[버니즈, 뉴진스, 하니, 모양, 이틀, 선택, 에이스, 마음, 하루, 하이브]",[하니]
280913,NewJeans,How Sweet,node,Ugz_CKnpd-xI8WPa0IN4AaABAg,NaN,뉴진스 민지 하니 다니엘 해린 혜인 민희진 대표님 모두 별일 없이 지내시고 웃으며...,@leViaThanNWJNS,2024-08-31T00:44:37Z,49,True,"[대표, 해린, 뉴진스, 하니, 별일, 응원, 민지, 혜인, 다니엘, 민희진]","[민지, 하니, 다니엘, 해린, 혜인]"


In [94]:
labeled = pd.read_csv('/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-NIPA-google-trendpop/가사 데이터/final_output/keyword_final/KPOP_total_comments_labeled.csv')

In [95]:
len(labeled) == len(sample_df)

True

In [96]:
labeled['sentiment_label'].head()

0    0
1    0
2    0
3    2
4    2
Name: sentiment_label, dtype: int64

In [97]:
sample_df['predicted_label'] = labeled['sentiment_label']

In [98]:
sample_df[['comment', 'predicted_label']][sample_df['predicted_label']==2].head(100)

,comment,predicted_label
3,이야 노래 종나 멋있네,2
4,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,2
6,해원이 외모체크 이후로 찐팬됨 멤버 모두 너무 착하고 예쁨 실력도 탑이네,2
7,와 진짜 설윤이랑 지우랑 비주얼 폭발하는 노래 하나 타이틀로 내자 정말 엔믹 요즘 ...,2
12,사랑해요,2
13,규진이 교정웃음 왜이리이쁘지? ㄷㄷ,2
15,소녀감성 다 필요없어.....평생 힙합해줘........,2
16,K팝 아티스트 최초 빌보드 라틴 뮤직 위크 참석 (유튜브로 직접 검색) 언제나 반전...,2
17,진짜 규진 설윤 너무 이쁘당,2
19,대박 대박 안 말하고 싶은데도 막 나오네요,2


In [ ]:
sample_df.to_csv('KPOP_comments_keyword_and_labels.csv')

In [4]:
import pandas as pd
filename = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard/keyword_list_after_cleaning_tfidf'
df = pd.read_csv(filename+'.csv')
df.head()

,Group,Title,keyword,weights
0,All,All,노래,"('노래', 0.021960686796069075)"
1,All,All,뮤비,"('뮤비', 0.016451318436040117)"
2,All,All,사랑해,"('사랑해', 0.015364906029126636)"
3,All,All,화이팅,"('화이팅', 0.243637042327725)"
4,All,All,응원,"('응원', 0.018174725445613975)"


In [17]:
result_df = df.groupby(['Group', 'Title','Member'])['keyword'].apply(list).reset_index()
result_df.to_csv(filename+'_listtype.csv')

In [5]:
result_df = df.groupby(['Group', 'Title'])['keyword'].apply(lambda x: x.head(5).tolist()).reset_index()
result_df.to_csv(filename+'_top5_listtype.csv')